In [10]:
team_members_names = ['إسراء ياسر ابوالقاسم',
                      'ايمان علاء فرج كامل',
                      'منة الله مصطفى مصطفى عوض',
                      'منة محيي الدين محمود',
                      'ميرنا محمد يسري']
team_members_seatnumbers = ['2016170080',
                            '2016170113'
                            '2016170437',
                            '2016170438',
                            '2016170450']

In [209]:
from tqdm import tqdm
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np
import glob
import math
from scipy import interpolate
from PIL import Image, ImageDraw
from scipy import ndimage
import cv2
import math 
import copy

# Hardcoded values

In [129]:
video_names = ['White Lane', 'Yello Lane']
roi_indices = np.array([[[130, 540], [900, 539], [550, 329], [419, 329]]], 'int32') # BL BR UR UL

# Region of Interest

In [96]:
def extract_roi(img, roi_indices): 
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    mask = np.zeros_like(img)
    cv2.fillPoly(mask, roi_indices, [255, 255, 255])
    return cv2.bitwise_and(mask, img)

# Color Conversion and Thresholding

In [12]:
def convert_rgb_to_grayscale(img):
    # This function will do color transform from RGB to Gray
    return cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

In [13]:
def convert_rgb_to_hsv(img):
    # This function will do color transform from RGB to HLS
    return cv2.cvtColor(img, cv2.COLOR_RGB2HLS)

In [14]:
def color_thresholding(img, low_threshold, high_threshold):
    # define the fixed values of the pixels
    strong = 255
    outimg = np.zeros((img.shape[0],img.shape[1]))

    # extract the indices of the pixels per condition for each channel
    in_i, in_j, in_ch = np.where((low_threshold <= img & (img <= high_threshold)))
    irrelevant_i, irrelevant_j, irrelevant_ch = np.where((img < low_threshold) | (img > high_threshold))

    # update the indices with the corresponding pixel value
    outimg[in_i, in_j] = strong
    outimg[irrelevant_i, irrelevant_j] = 0

    return outimg

# Canny

In [15]:
def gaussian_kernel(sigma=0.5):
    # Specify the kernel size
    if sigma > 1:
        size = 5
    else:
        size = 3

    # Calculate the formula
    size = int(size) // 2
    x, y = np.mgrid[-size:size+1, -size:size+1]
    normal = 1 / (2.0 * np.pi * sigma**2)
    g = normal * np.exp(-((x**2 + y**2) / (2.0*sigma**2)))

    return g

In [ ]:
def convolve(input_arr, weights, c_type='conv', mode='reflect'):
    """
    Two-dimensional convolution.
    
    The array is convolved with the given kernel.
    
    Parameters
    -----------
    input_arr : array_like
        Input array to filter.
    
    weights : array_like
        Array of weights(kernel), same number of dimensions as input.
        
    c_type : {'conv', 'loop'}, optional
        The `c_type` parameter determines the implementation of the
        convolution.
        
        'conv'(default)
            Use built-in convolve.
        
        'loop'
            Use implemented loop.
        
    mode : {'constant', 'reflect'}, optional
        The `mode` parameter determines how the array borders are
        handled.
        
        'constant'
            Pads with a constant value.
        
        'reflect'(default)
            Pads with the reflection of the vector mirrored on the first and last values of the vector along each axis.
            
    Returns
    -------
    img : ndarray
        The result of convolution of `input` with `weights`.
    """
    img = copy.deepcopy(input_arr)
    if c_type == 'loop':
        window_size = weights.shape[0]
        offset = window_size//2
        # Pad the working image
        padded_img = np.pad(img, offset, mode=mode)
        height = padded_img.shape[0]
        width = padded_img.shape[1]
        img_temp = np.zeros((height, width))
        # The filter algorithm
        for i in range(offset, height - offset):
            for j in range(offset, width - offset):
                img_temp[i, j] = np.sum(np.multiply(padded_img[i-offset: i+offset+1, j-offset: j+offset+1], weights))
        img = img_temp[offset:height-offset, offset:width-offset]    
    else:
        img = ndimage.filters.convolve(input_arr, weights, mode=mode)
    return img

In [16]:
def noise_reduction(img, c_type='conv', mode='reflect'):
    kernel = gaussian_kernel(1.4)
    return convolve(img, kernel, c_type=c_type, mode=mode)

In [17]:
def gradient_calculation(img, c_type='conv', mode='reflect'):
    Kx = np.array([[-1, 0, 1], [-2, 0, 2], [-1, 0, 1]], dtype=np.float32)
    Ky = np.array([[1, 2, 1], [0, 0, 0], [-1, -2, -1]], dtype=np.float32)
    
    Ix = convolve(img, Kx, c_type=c_type, mode=mode)
    Iy = convolve(img, Ky, c_type=c_type, mode=mode)
    
    G = np.hypot(Ix, Iy)
    G = G / G.max() * 255
    theta = np.arctan2(Iy, Ix)

    G = G.astype(int)
    return G, theta

In [200]:
def non_max_suppression(img, theta):
    height, width = img.shape
    out = np.zeros((height, width), dtype=int)
    # Change from radian to degree
    angle = theta * 180 / np.pi
    angle[angle < 0] += 180

    for i in range(1, height-1):
        for j in range(1, width-1):
            point_1 = 255
            point_2 = 255

            # Angle 0°
            # gradient magnitude is greater than the magnitudes at pixels in the (east and west) directions
            if (0 <= angle[i, j] < 22.5) or (157.5 <= angle[i, j] <= 180):
                point_1 = img[i, j-1]
                point_2 = img[i, j+1]
            # Angle 45°
            # gradient magnitude is greater than the magnitudes at pixels in the (north east and south west) directions.
            elif 22.5 <= angle[i, j] < 67.5:
                point_1 = img[i-1, j+1]
                point_2 = img[i+1, j-1]
            # Angle 90°
            # gradient magnitude is greater than the magnitudes at pixels in the (north and south) directions.
            elif 67.5 <= angle[i, j] < 112.5:
                point_1 = img[i-1, j]
                point_2 = img[i+1, j]
            # Angle 135°
            # gradient magnitude is greater than the magnitudes at pixels in the (north west and south-east) directions.
            elif 112.5 <= angle[i, j] < 157.5:
                point_1 = img[i-1, j-1]
                point_2 = img[i+1, j+1]

            # if the pixel's value is the maximum then update the out, otherwise leave it with value = 0
            if (img[i, j] >= point_1) and (img[i, j] >= point_2):
                out[i, j] = img[i, j]
    return out

In [19]:
def double_thresholding(img, low_threshold=2, high_threshold=25):
    # define the fixed values of the pixels
    weak = 25
    strong = 255

    # extract the indices of the pixels per condition
    strong_i, strong_j = np.where(img >= high_threshold)
    weak_i, weak_j = np.where((low_threshold <= img) & (img <= high_threshold))
    irrelevant_i, irrelevant_j = np.where(img < low_threshold)

    # update the indices with the corresponding pixel value
    img[strong_i, strong_j] = strong
    img[weak_i, weak_j] = weak
    img[irrelevant_i, irrelevant_j] = 0

    return img, weak, strong

In [20]:
def hysteresis_edge_tracking(img, weak, strong):
    height, width = img.shape
    # Get the indices of the weak pixels
    weak_i, weak_j = np.where(img == weak)
    for (i, j) in zip(weak_i, weak_j):
        if i == 0 or j == 0 or i == height - 1 or j == width - 1:
            continue
        # Check if any of the  8-connected neighborhood pixels is a strong pixel
        if np.any(np.where(img[i-1:i+2, j-1:j+2] == strong)):
            img[i, j] = strong
        # If not, then suppress the weak pixel
        else:
            img[i, j] = 0
    return img

In [21]:
# Helper Function, to reduce the redundant
# plotting code
def plotting(img, fig_num, title):
    plt.figure(fig_num)
    plt.imshow(img, cmap='gray')
    plt.title(title)
    return

In [22]:
def detect_edges_canny(img, low_threshold=2, high_threshold=50, c_type='conv', mode='reflect'):
    """
    Arguments:
        :param img: np.ndarray((height, width)), gray-image
        :param low_threshold: an integer
        :param high_threshold: an integer
    :return: edges image
    """
    img = img.copy()

    # 1. Noise reduction
    # using Gaussian Smoothing Kernel
    img = noise_reduction(img, c_type=c_type, mode=mode)
    plotting(img, 1, 'Gaussian Filter')

    # 2. Gradient Calculations
    # using Sobel kernels
    img, theta = gradient_calculation(img, c_type=c_type, mode=mode)
    plotting(img, 2, 'Sobel Filter (G)')

    # 3. Non-Maximum Suppression
    img = non_max_suppression(img, theta)
    plotting(img, 3, 'Non-Maximum Suppression')

    # 4. Double Thresholding
    img, weak, strong = double_thresholding(img, low_threshold, high_threshold)
    plotting(img, 4, 'Double Thresholding')

    # 5. Edge Tracking by Hysteresis
    img = hysteresis_edge_tracking(img, weak, strong)
    plotting(img, 5, 'Edge Tracking by Hysteresis')

    plt.show()
    return img

# Remove Guassian Noise

In [23]:
def remove_noise(img, kernel_size, width, height):
    # You should implement Gaussian Noise Removal Here
    #Mean_filter
    #height = img.shape(0)
    #width = img.shape(1)
    final_image = img.copy()
    mask_one_dim = math.sqrt(kernel_size)
    offset = int(mask_one_dim // 2)
    for i in range(offset, height - offset):
        for j in range(offset, width - offset):
            sum = 0
            for r in range(i - offset, i + offset + 1):
                for c in range(j - offset, j + offset + 1):
                    sum += img[r][c]
            final_image[i][j] = sum // kernel_size
    return final_image

# Masking Image

In [204]:
def mask_image(img, vertices, width, height):
    # Mask out the pixels outside the region defined in vertices (set the color to black)
    out_img = img.copy();
    '''for i in range(0, height):
        for j in range(0, width):
            if vertices[i][j] == 0:
                out_img[i][j] = 0
    return out_img'''
    return cv2.bitwise_and(out_img, vertices)

# Hough Transform

In [183]:
# Apply Hough transform to find the lanes
# Input:
#   img - numpy array: binary image containing the edges after applying Canny Detector
# Output:
#   lines - list of list: list of lines, each line = [x1, y1, x2, y2]
def hough_transform(img, accepted_ratio=0.1, rho_step=1, theta_step=1):
    #print('\nHough Transform in progress')
    # Calculating diagonal length of the image to define the range of the rhos
    height, width = img.shape
    diagonal_length = round(math.sqrt(height ** 2 + width ** 2))
    rhos = np.arange(-diagonal_length, diagonal_length, step=rho_step)

    # Setting the thetas and their cosines/sines
    thetas = np.arange(-90.0, 90.0, step=theta_step)
    cos_thetas = np.cos(np.deg2rad(thetas))
    sin_thetas = np.sin(np.deg2rad(thetas))

    accumulator = np.zeros((len(rhos), len(thetas)))

    edge_pts_ys, edge_pts_xs = np.nonzero(img)
    for i in range(len(edge_pts_xs)):
        x = edge_pts_xs[i]
        y = edge_pts_ys[i]
        for theta_idx in range(len(thetas)):
            rho = x * cos_thetas[theta_idx] + y * sin_thetas[theta_idx]
            rho_idx = np.argmin(np.abs(rhos - rho))  # find the index of the closest rho to the computed rho
            accumulator[rho_idx, theta_idx] += 1

    # lines_rhos = []
    # lines_thetas = []
    lines_rhos_indices, lines_thetas_indices = np.nonzero(accumulator >= accumulator.max() * accepted_ratio)
    lines = []
    for i in range(len(lines_rhos_indices)):
        rho_idx = lines_rhos_indices[i]
        rho = rhos[rho_idx]
        # lines_rhos.append(rho)

        theta_idx = lines_thetas_indices[i]
        theta = thetas[theta_idx]
        # lines_thetas.append(theta)

        a = cos_thetas[theta_idx]
        b = sin_thetas[theta_idx]
        x0 = a * rho
        y0 = b * rho
        x1 = int(x0 + 10000 * (-b))
        y1 = int(y0 + 10000 * a)
        x2 = int(x0 - 10000 * (-b))
        y2 = int(y0 - 10000 * a)
        lines.append([x1, y1, x2, y2])
    return lines

In [26]:
def draw_hough_lines(img, lines, path, color=[255, 0, 0], thickness=8):
    d = ImageDraw.Draw(img)
    for line in lines:
        #d.line(line, fill=(255, 255, 255), width=thickness)
        d.line(line, fill=(color[0], color[1], color[2]), width=thickness)
    #img.save(path)
    return img

In [27]:
def hough_test():
    input_image = read_image('Images/hough_test_1.jpg')
    edges = detect_edges_canny(input_image, 2, 50)
    lines = hough_transform(edges, 0.3)
    draw_hough_lines(Image.open('Images/hough_test_1.jpg'), lines, 'Images/hough_test_1_out.jpg')

    input_image = read_image('Images/hough_test_2.jpg')
    edges = detect_edges_canny(input_image, 2, 50)
    lines = hough_transform(edges)
    draw_hough_lines(Image.open('Images/hough_test_2.jpg'), lines, 'Images/hough_test_2_out.jpg')
    return

# Getting Lane Lines

In [28]:
def get_line_length(line):
    x1, y1, x2, y2 = line
    return math.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

In [29]:
# y = mx + b
# This function returns the slope (m) and the intercept (b) of the input line.
def get_line_slope_intercept(line):
    x1, y1, x2, y2 = line
    if x2 - x1 == 0:
        return math.inf, 0
    slope = (y2 - y1) / (x2 - x1)
    # b = y - mx
    intercept = y1 - slope * x1
    return slope, intercept

In [30]:
# The upper left corner of the image is (0, 0)
# X increases left-right
# Y increases top-down
# Left line: as Y increases, X decreases, so it has a negative slope
# Right line: as Y increase, X increases, so it has a positive slope

# Input: a list of lines, where each line = [x1, y1, x2, y2]
# Output: avg_of_left_lines, avg_of_right_lines, each is a tuple(avg_slope, avg_intercept)
def get_avg_slope_intercept(lines):
    right_lines = []
    right_lengths = []
    left_lines = []
    left_lengths = []
    for line in lines:
        slope, intercept = get_line_slope_intercept(line)
        if slope == math.inf or (slope >= -0.4 and slope <= 0.4):
            continue
        line_length = get_line_length(line)
        if slope < 0: # left line
            left_lines.append([slope, intercept])
            left_lengths.append(line_length)
        else:         # right line
            right_lines.append([slope, intercept])
            right_lengths.append(line_length)

    # Weighted average of all right lines
    if len(right_lines) > 0:
        avg_of_right_lines = np.dot(right_lengths, right_lines) / np.sum(right_lengths)
    else:
        avg_of_right_lines = None
    
    # Weighted average of all left lines 
    if len(left_lines) > 0:
        avg_of_left_lines = np.dot(left_lengths, left_lines) / np.sum(left_lengths)
    else:
        avg_of_left_lines = None
    
    return avg_of_right_lines, avg_of_left_lines

In [31]:
# Input: y1, y2, line in the form (slope, intercept)
# Output: line in the form [x1, y1, x2, y2]
# y = mx + b, x = (y - b) / m
def get_line_endpoints(y1, y2, line):
    if line is None:
        return None
    slope, intercept = line
    x1 = int((y1 - intercept) / slope)
    y1 = int(y1)
    x2 = int((y2 - intercept) / slope)
    y2 = int(y2)
    return [x1, y1, x2, y2]

In [32]:
# Hough returns a lot of lines for one lane line, so we average them to have one line per lane line.
def get_lane_lines(img, lines):
    avg_of_left_lines, avg_of_right_lines = get_avg_slope_intercept(lines)
    y1 = img.shape[0] 
    y2 = img.shape[0] * 0.59
    left_lane = get_line_endpoints(y1, y2, avg_of_left_lines)
    right_lane = get_line_endpoints(y1, y2, avg_of_right_lines)
    return left_lane, right_lane

In [181]:
def draw_lines_connected(img, hough_lines, color=[0, 0, 255], thickness=8):
    # This function should draw lines to the images (default color is red and thickness is 8)
    lane_lines = get_lane_lines(img, hough_lines)
    #lane_lines = hough_lines
    lines_image = np.zeros_like(img)
    for line in lane_lines:
        if line is None:
            continue
        x1, y1, x2, y2 = line
        point_one = (line[0], line[1])
        point_two = (line[2], line[3])
        cv2.line(lines_image, point_one, point_two, color, thickness)
    final_image = cv2.addWeighted(img, 0.8, lines_image, beta=0.95, gamma=0)
    #cv2.imshow("image with lines", final_image)
    #cv2.waitKey(0)
    return final_image

# I/O

In [35]:
def read_image(img_path):
    #img = Image.open(img_path)
    #img = img.convert("L")
    #img = np.asarray(img, dtype=int)
    return mpimg.imread(img_path)

In [36]:
def read_video(video_path):
    video = cv2.VideoCapture(video_path)
    fps = video.get(cv2.CAP_PROP_FPS)
    return video, fps

In [166]:
def get_video_frames(video):
    frames = []
    width = int(video.get(3))
    height = int(video.get(4))
    number_of_frames = int(video.get(7))
    for i in range(number_of_frames):
        success, frame = video.read()
        if success:
            frames.append(frame)
    return frames, width, height

In [160]:
def write_video(pathOut, frames, fps, width, height):
    size = (width, height)
    out = cv2.VideoWriter(pathOut, cv2.VideoWriter_fourcc(*'DIVX'), fps, size)
    for frame in frames:
        out.write(frame)
    out.release()

# Pipelines

In [206]:
def image_pipeline(image, width, height, video_frame_flag):    
    if not video_frame_flag:
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    original_image = image
    
    # extract region of interest
    image = extract_roi(image, roi_indices)

    hsv_img = convert_rgb_to_hsv(image)

    # 3. convert to Gray
    gray_img = convert_rgb_to_grayscale(image)

    # 4. Threshold HSV for Yellow and White (combine the two results together)

    # 4.1 Threshold HSV for Yellow
    # lower = np.uint8([10, 0, 200])
    # upper = np.uint8([40, 255, 255])
    lower = np.uint8([10, 0, 100])
    upper = np.uint8([40, 255, 255])
    yellow_mask = color_thresholding(hsv_img, lower, upper)

    # 4.1 Threshold HSV for White
    # lower = np.uint8([0, 0, 200])
    # upper = np.uint8([255, 55, 255])
    lower = np.uint8([0, 200, 0])
    upper = np.uint8([200, 255, 255])
    white_mask = color_thresholding(hsv_img, lower, upper)

    # 4.1 Combine the resuls together
    image = cv2.bitwise_or(white_mask, yellow_mask)

    # 5. Mask the gray image using the threshold output from step 4
    #masked_gray_img = mask_image(gray_img, mask, width, height)

    # 6. Apply noise remove (gaussian) to the masked gray image
    #masked_gray_img_no_noise = remove_noise(image, 3, width, height)

    # 7. use canny detector and fine tune the thresholds (low and high values)
    # TODO: masked_gray_img should be of data-type numpy.ndarray
    canny_out_img = detect_edges_canny(image, low_threshold=10, high_threshold=50)

    # 8. mask the image using the canny detector output
    masked_with_canny = mask_image(image, canny_out_img, width, height)

    # 9. apply hough transform to find the lanes
    hough_lines = hough_transform(masked_with_canny)

    # 10. apply the pipeline you developed to the challenge videos
    image_with_lines = draw_lines_connected(original_image, hough_lines)

    return image_with_lines

In [207]:
def video_pipeline(video_frames, width, height, fps, name):
    new_video_frames = []
    for frame in tqdm(video_frames):
        new_frame = image_pipeline(frame, width, height)
        new_video_frames.append(new_frame)
    write_video('Videos/' + name + " Output.mp4", new_video_frames, fps, width, height)
    print("Video saved succesffuly")

# MAIN

In [208]:
for video_name in video_names:
    print(video_name)
    video, fps = read_video('Videos/' + video_name + '.mp4')
    video_frames, width, height = get_video_frames(video)
    video_pipeline(video_frames, width, height, fps, video_name)

White Lane






  0%|                                                  | 0/214 [00:00<?, ?it/s]



  0%|▏                                         | 1/214 [00:05<18:07,  5.11s/it]



  1%|▍                                         | 2/214 [00:10<18:15,  5.17s/it]



  1%|▌                                         | 3/214 [00:15<17:55,  5.10s/it]



  2%|▊                                         | 4/214 [00:20<17:22,  4.96s/it]



  2%|▉                                         | 5/214 [00:24<16:28,  4.73s/it]



  3%|█▏                                        | 6/214 [00:28<15:48,  4.56s/it]



  3%|█▎                                        | 7/214 [00:32<15:45,  4.57s/it]



  4%|█▌                                        | 8/214 [00:37<15:25,  4.49s/it]



  4%|█▊                                        | 9/214 [00:41<15:30,  4.54s/it]



  5%|█▉                                       | 10/214 [00:46<15:28,  4.55s/it]



  5%|██                                       | 11/214 [00:51<15:31,  4.59s/it]





 80%|████████████████████████████████▏       | 172/214 [13:18<03:17,  4.70s/it]



 81%|████████████████████████████████▎       | 173/214 [13:22<03:10,  4.65s/it]



 81%|████████████████████████████████▌       | 174/214 [13:26<02:56,  4.42s/it]



 82%|████████████████████████████████▋       | 175/214 [13:30<02:48,  4.31s/it]



 82%|████████████████████████████████▉       | 176/214 [13:34<02:40,  4.23s/it]



 83%|█████████████████████████████████       | 177/214 [13:39<02:36,  4.24s/it]



 83%|█████████████████████████████████▎      | 178/214 [13:43<02:33,  4.26s/it]



 84%|█████████████████████████████████▍      | 179/214 [13:47<02:29,  4.27s/it]



 84%|█████████████████████████████████▋      | 180/214 [13:52<02:27,  4.35s/it]



 85%|█████████████████████████████████▊      | 181/214 [13:56<02:27,  4.46s/it]



 85%|██████████████████████████████████      | 182/214 [14:02<02:30,  4.69s/it]



 86%|██████████████████████████████████▏     | 183/214 [14:07<02:29,  4.83s/it]



 86%

Video saved succesffuly
Yello Lane






  0%|                                                  | 0/674 [00:00<?, ?it/s]



  0%|                                          | 1/674 [00:04<49:20,  4.40s/it]



  0%|                                          | 2/674 [00:09<50:35,  4.52s/it]



  0%|▏                                         | 3/674 [00:13<51:26,  4.60s/it]



  1%|▏                                         | 4/674 [00:18<50:59,  4.57s/it]



  1%|▎                                         | 5/674 [00:22<50:27,  4.52s/it]



  1%|▎                                         | 6/674 [00:26<48:25,  4.35s/it]



  1%|▍                                         | 7/674 [00:30<47:25,  4.27s/it]



  1%|▍                                         | 8/674 [00:34<46:15,  4.17s/it]



  1%|▌                                         | 9/674 [00:38<45:47,  4.13s/it]



  1%|▌                                        | 10/674 [00:43<45:57,  4.15s/it]



  2%|▋                                        | 11/674 [00:47<46:22,  4.20s/it]





 26%|██████████▏                             | 172/674 [12:46<35:19,  4.22s/it]



 26%|██████████▎                             | 173/674 [12:50<35:22,  4.24s/it]



 26%|██████████▎                             | 174/674 [12:54<35:25,  4.25s/it]



 26%|██████████▍                             | 175/674 [12:59<35:28,  4.26s/it]



 26%|██████████▍                             | 176/674 [13:03<35:47,  4.31s/it]



 26%|██████████▌                             | 177/674 [13:08<36:38,  4.42s/it]



 26%|██████████▌                             | 178/674 [13:13<39:02,  4.72s/it]



 27%|██████████▌                             | 179/674 [13:18<40:09,  4.87s/it]



 27%|██████████▋                             | 180/674 [13:23<39:42,  4.82s/it]



 27%|██████████▋                             | 181/674 [13:28<38:33,  4.69s/it]



 27%|██████████▊                             | 182/674 [13:31<36:27,  4.45s/it]



 27%|██████████▊                             | 183/674 [13:35<34:55,  4.27s/it]



 27%

 51%|████████████████████▍                   | 344/674 [25:17<23:09,  4.21s/it]



 51%|████████████████████▍                   | 345/674 [25:20<22:20,  4.07s/it]



 51%|████████████████████▌                   | 346/674 [25:24<22:06,  4.05s/it]



 51%|████████████████████▌                   | 347/674 [25:28<22:01,  4.04s/it]



 52%|████████████████████▋                   | 348/674 [25:32<21:56,  4.04s/it]



 52%|████████████████████▋                   | 349/674 [25:37<22:04,  4.08s/it]



 52%|████████████████████▊                   | 350/674 [25:41<22:25,  4.15s/it]



 52%|████████████████████▊                   | 351/674 [25:45<22:53,  4.25s/it]



 52%|████████████████████▉                   | 352/674 [25:50<23:41,  4.41s/it]



 52%|████████████████████▉                   | 353/674 [25:55<24:23,  4.56s/it]



 53%|█████████████████████                   | 354/674 [26:00<24:24,  4.58s/it]



 53%|█████████████████████                   | 355/674 [26:04<23:48,  4.48s/it]



 53%

 77%|██████████████████████████████▌         | 516/674 [37:59<12:00,  4.56s/it]



 77%|██████████████████████████████▋         | 517/674 [38:03<11:38,  4.45s/it]



 77%|██████████████████████████████▋         | 518/674 [38:07<11:12,  4.31s/it]



 77%|██████████████████████████████▊         | 519/674 [38:12<11:05,  4.29s/it]



 77%|██████████████████████████████▊         | 520/674 [38:16<11:07,  4.34s/it]



 77%|██████████████████████████████▉         | 521/674 [38:20<10:59,  4.31s/it]



 77%|██████████████████████████████▉         | 522/674 [38:25<11:03,  4.36s/it]



 78%|███████████████████████████████         | 523/674 [38:29<11:11,  4.45s/it]



 78%|███████████████████████████████         | 524/674 [38:34<11:35,  4.64s/it]



 78%|███████████████████████████████▏        | 525/674 [38:40<11:54,  4.79s/it]



 78%|███████████████████████████████▏        | 526/674 [38:45<11:56,  4.84s/it]



 78%|███████████████████████████████▎        | 527/674 [38:49<11:45,  4.80s/it]



 78%

Video saved succesffuly
